Welcome to Wmfdata-Python! This notebook gives a introduction to using it.

If you have additional questions, please ask in the [#product-analytics channel](https://app.slack.com/client/T024KLHS4/CLKDS4MG9) on the private Wikimedia Foundation Slack or the public [analytics mailing list](mailto:analytics@lists.wikimedia.org).

## Contents
1. [Importing](#importing)
1. [Docstrings](#docstrings)
1. [Presto](#presto)
1. [Spark](#spark)
1. [MariaDB](#mariadb)
1. [Hive](#hive)
1. [Utils](#utils)
   1. [`pd_display_all`](#pd_display_all)
   1. [`insert_code_toggle`](#insert_code_toggle)
   1. [`df_to_remarkup`](#df_to_remarkup)
   1. [`get_dblist`](#get_dblist)
   1. [`sql_tuple`](#sql_tuple)

## Importing

The standard way to import Wmfdata gives it the alias `wmf`.

In [1]:
import wmfdata as wmf

This automatically imports Wmfdata's six modules, so they are now available as:
* `wmf.mariadb`: running SQL on the [Analytics MediaWiki replicas](https://wikitech.wikimedia.org/wiki/Analytics/Systems/MariaDB) using MariaDB
* `wmf.presto`: running SQL on the Data Lake using [Presto](https://wikitech.wikimedia.org/wiki/Analytics/Systems/Presto)
* `wmf.spark`: running SQL on the Data Lake using [Spark](https://wikitech.wikimedia.org/wiki/Analytics/Systems/Cluster/Spark)
* `wmf.hive`: running SQL on the Data Lake using [Hive](https://wikitech.wikimedia.org/wiki/Analytics/Systems/Cluster/Hive)
* `wmf.utils`: miscellaneous utilities
* `wmf.conda`: tools for Conda environments; mainly for internal use.

In [2]:
wmf.hive

<module 'wmfdata.hive' from '/home/neilpquinn-wmf/wmfdata-python/wmfdata/hive.py'>

## Docstrings

IPython (the Python shell used by Jupyter) allows you to access the docstring of functions and modules by appending `?` to the end of their names. Wmfdata's key functions are all documented this way, so we'll use that trick in this guide.

## Presto

`wmf.presto` has just one function: `run`. 

In [3]:
wmf.presto.run?

Signature: wmf.presto.run(commands, catalog='analytics_hive')
Docstring:
Runs one or more SQL commands using the Presto SQL engine and returns the last result
in a Pandas dataframe.

Presto can` be connected to many different backend data catalogs. Currently, it is only connected to the Data Lake, which has the catalog name "analytics_hive".

Arguments:
* `commands`: the SQL to run. A string for a single command or a list of
  strings for multiple commands within the same session (useful for things
  like setting session variables). Passing more than one query is not
  supported; only results from the second will be returned.
File:      ~/wmfdata-python/wmfdata/presto.py
Type:      function


In [4]:
countries = wmf.presto.run("""
SELECT *
FROM canonical_data.countries
""")

Like all the other `run` functions, `wmf.presto.run` returns its results as a [Pandas](https://pandas.pydata.org/docs/) dataframe.

In [5]:
countries

,name,iso_code,economic_region,maxmind_continent
0,Afghanistan,AF,Global South,Asia
1,Åland Islands,AX,Global North,Europe
2,Albania,AL,Global North,Europe
3,Algeria,DZ,Global South,Africa
4,American Samoa,AS,Global South,Oceania
...,...,...,...,...
246,Western Sahara,EH,Global South,Africa
247,Yemen,YE,Global South,Asia
248,Zambia,ZM,Global South,Africa
249,Zimbabwe,ZW,Global South,Africa


In [6]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               251 non-null    object
 1   iso_code           251 non-null    object
 2   economic_region    251 non-null    object
 3   maxmind_continent  251 non-null    object
dtypes: object(4)
memory usage: 8.0+ KB


## Spark

Similarly, `wmf.spark` also has a `run` function for running SQL.

In [7]:
wmf.spark.run?

Signature: wmf.spark.run(commands: Union[str, List[str]]) -> pandas.core.frame.DataFrame
Docstring:
Runs SQL commands against the Hive tables in the Data Lake using the
PySpark SQL interface.

Note: this command will use the existing Spark session if there is one and
otherwise create a predefined "yarn-regular" session. If you want to use
a different type of session, use `create_session` or `create_custom_session`
first.

Note: this function loads all the output into memory on the client. If
your command produces many gigabytes of output, it could cause an
out-of-memory error.

Arguments:
* `commands`: the SQL to run. A string for a single command or a list of
  strings for multiple commands within the same session (useful for things
  like setting session variables). Passing more than one query is *not*
  supported; only results from the second will be returned.
File:      ~/wmfdata-python/wmfdata/spark.py
Type:      function


Unfortunately, Spark produces a lot of logspam when a session is being created. The messages here are all normal and should just be ignored.

In [12]:
countries = wmf.spark.run("""
SELECT *
FROM canonical_data.countries
""")

countries

,name,iso_code,economic_region,maxmind_continent
0,Afghanistan,AF,Global South,Asia
1,Åland Islands,AX,Global North,Europe
2,Albania,AL,Global North,Europe
3,Algeria,DZ,Global South,Africa
4,American Samoa,AS,Global South,Oceania
...,...,...,...,...
246,Western Sahara,EH,Global South,Africa
247,Yemen,YE,Global South,Asia
248,Zambia,ZM,Global South,Africa
249,Zimbabwe,ZW,Global South,Africa


If you want to use the [PySpark API](https://spark.apache.org/docs/2.4.4/api/python/pyspark.html) directly, the `spark.create_session` function makes it easy to create a Spark session.

In [7]:
wmf.spark.create_session?

Signature:
wmf.spark.create_session(
    type='yarn-regular',
    app_name=None,
    extra_settings={},
    ship_python_env=False,
)
Docstring:
Creates a new Spark session based on one of the PREDEFINED_SPARK_SESSION
types, stopping any existing session first.

Arguments:
* `type`: the type of Spark session to create.
    * "local": Run the command in a local Spark process. Use this for
      prototyping or querying small-ish data (less than a couple of GB).
    * "yarn-regular": the default; able to use up to 15% of Hadoop cluster
      resources (This is the default).
    * "yarn-large": for queries which require more processing (e.g. joins) or
      which access more data; able to use up to 30% of Hadoop cluster
      resources.
* `extra_settings`: A dict of additional Spark configs to use when creating
  the Spark session. These will override the defaults specified
  by `type`.
* `ship_python_env`: If master='yarn' and this is True, a conda env will be packed
  and shipped to remot

In [8]:
ss = wmf.spark.create_session()

22/11/22 02:26:24 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/11/22 02:26:24 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
22/11/22 02:26:24 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
22/11/22 02:26:24 WARN Utils: Service 'sparkDriver' could not bind on port 12002. Attempting port 12003.
22/11/22 02:26:24 WARN Utils: Service 'sparkDriver' could not bind on port 12003. Attempting port 12004.
22/11/22 02:26:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/22 02:26:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/22 02:26:24 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/11/22 02:26:24 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attemptin

In [9]:
ss

`spark.create_custom_session` allows you to create a Spark session with complete control over all the settings.

In [10]:
wmf.spark.create_custom_session?

Signature:
wmf.spark.create_custom_session(
    master='local[2]',
    app_name='wmfdata-custom',
    spark_config={},
    ship_python_env=False,
    conda_pack_kwargs={},
)
Docstring:
Creates a new Spark session, stopping any existing session first.

Use this instead of create_session if you'd rather have manual control over
your SparkSession configuration.

Arguments:
* `master`: passed to SparkSession.builder.master()
  If this is "yarn" and and a conda env is active and and ship_python_env=False,
  remote executors will be configured to use conda.conda_base_env_prefix(),
  which for Spark 2 defaults to anaconda-wmf and for Spark 3 defalts to conda-analytics.
  This should usually work as both are installed on all WMF YARN worker nodes.
  If your conda environment has required packages installed that are not in those, set
  ship_python_env=True.
* `app_name`: passed to SparkSession.builder.appName().
* `spark_config`: passed to SparkSession.builder.config()
* `ship_python_env`: If m

In [11]:
ss = wmf.spark.create_custom_session(
    master="yarn",
    spark_config={
        "spark.driver.memory": "2g",
        "spark.dynamicAllocation.maxExecutors": 64,
        "spark.executor.memory": "8g",
        "spark.executor.cores": 4,
        "spark.sql.shuffle.partitions": 256
    }
)

22/11/22 02:26:31 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/11/22 02:26:31 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
22/11/22 02:26:31 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
22/11/22 02:26:31 WARN Utils: Service 'sparkDriver' could not bind on port 12002. Attempting port 12003.
22/11/22 02:26:31 WARN Utils: Service 'sparkDriver' could not bind on port 12003. Attempting port 12004.
22/11/22 02:26:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/22 02:26:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/22 02:26:31 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/11/22 02:26:31 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attemptin

In [12]:
ss

## MariaDB

`wmf.mariadb.run` allows you to run SQL queries using MariaDB. Unlike the other `run` functions, which access the same data in the [Data Lake](https://wikitech.wikimedia.org/wiki/Analytics/Data_Lake), this accesses a different set of data in the [Analytics MediaWiki replicas](https://wikitech.wikimedia.org/wiki/Analytics/Systems/MariaDB). 

In [16]:
wmf.mariadb.run?

Signature:
wmf.mariadb.run(
    commands,
    dbs,
    use_x1=False,
    format='pandas',
    date_col=None,
    index_col=None,
)
Docstring:
Run SQL queries or commands on the Analytics MediaWiki replicas.

Arguments:
* `commands`: the SQL to run. A string for a single command or a list of
  strings for multiple commands within the same session (useful for things
  like setting session variables).
* `dbs`: a string for one database or a list to run the commands on
  multiple databases and concatenate the results.  Possible values:
    * a wiki's database code (e.g. "enwiki", "arwiktionary", "wikidatawiki")
      for its MediaWiki database (or its ExtensionStorage database if
      `use_x1` is passed)
    * "logs" for the EventLogging
    * "centralauth" for global accounts
    * "wikishared" for cross-wiki ExtensionStorage
    * "staging" for user-writable ad-hoc tests and analysis
* `use_x1`: whether to the connect to the given database on the
  ExtensionStorage replica (only works f

In [17]:
wmf.mariadb.run(
    """
    SELECT *
    FROM page
    LIMIT 10
    """,
    "enwiki"
)

,page_id,page_namespace,page_title,page_is_redirect,page_is_new,page_random,page_touched,page_links_updated,page_latest,page_len,page_content_model,page_lang
0,10,0,AccessibleComputing,1,0,0.331671,20221115041429,20221023043017,1002250816,111,wikitext,None
1,12,0,Anarchism,0,0,0.786172,20221121042644,20221121042656,1122800568,109009,wikitext,None
2,13,0,AfghanistanHistory,1,0,0.062150,20221112062329,20221023043017,783865149,90,wikitext,None
3,14,0,AfghanistanGeography,1,0,0.952234,20221023042651,20221023043017,783865160,92,wikitext,None
4,15,0,AfghanistanPeople,1,0,0.574721,20221106222659,20221023043017,783865293,95,wikitext,None
5,18,0,AfghanistanCommunications,1,0,0.751068,20221023042651,20221023043017,783865299,97,wikitext,None
6,19,0,AfghanistanTransportations,1,0,0.674273,20221023042651,20221023043017,783821589,113,wikitext,None
7,20,0,AfghanistanMilitary,1,0,0.118158,20221118025602,20221023043017,1093067805,154,wikitext,None
8,21,0,AfghanistanTransnationalIssues,1,0,0.567973,20221031093955,20221023043017,783821743,101,wikitext,None
9,23,0,AssistiveTechnology,1,0,0.723041,20221107141956,20221023043017,783865310,88,wikitext,None


Here's an example showing some of the function's fancier features. It's running the query against five different databases and combining the results (note that you have to use the `DATABASE` function in the query so you can tell which results came from which wiki). It also parsing the `user_registration` column from a MediaWiki date string (e.g. `"20100915021055"`) to a proper Pandas datetime and then setting the `user_id` column as the index of the dataframe.

In [39]:
users = wmf.mariadb.run(
    """
    SELECT
        DATABASE() AS wiki,
        user_name,
        user_id,
        user_registration
    FROM user
    WHERE user_name IN ('Neil Shah-Quinn', 'Neil Shah-Quinn (WMF)')
    """,
    dbs=["enwiki", "eswiki", "frwiki", "arwiki", "jawiki"],
    date_col="user_registration",
    index_col="user_id"
)

users

,wiki,user_name,user_registration
user_id,,,
13076609,enwiki,Neil Shah-Quinn,2010-09-15 02:10:55
24753008,enwiki,Neil Shah-Quinn (WMF),2015-04-13 15:45:38
1726054,eswiki,Neil Shah-Quinn,2011-02-04 23:06:08
3643219,eswiki,Neil Shah-Quinn (WMF),2015-04-16 17:21:43
1533207,frwiki,Neil Shah-Quinn,2013-04-07 19:35:02
2194938,frwiki,Neil Shah-Quinn (WMF),2015-04-20 17:43:18
311761,arwiki,Neil Shah-Quinn,2010-09-20 16:00:11
981709,arwiki,Neil Shah-Quinn (WMF),2015-04-17 22:09:46
701248,jawiki,Neil Shah-Quinn,2013-01-24 02:55:15


## Hive

`wmf.hive.run` allows you to run SQL using Hive, although in general it's better to use Presto or Spark to query the Data Lake instead.

In [9]:
wmf.hive.run?

Signature: wmf.hive.run(commands)
Docstring:
Runs SQL commands against the Hive tables in the Data Lake.

Arguments:
* `commands`: the SQL to run. A string for a single command or a list of
  strings for multiple commands within the same session (useful for things
  like setting session variables). Passing more than one query is *not*
  supported, and will usually result in an error.
File:      ~/wmfdata-python/wmfdata/hive.py
Type:      function


In [11]:
countries = wmf.hive.run(f"""
SELECT *
FROM canonical_data.countries
""")

countries

,name,iso_code,economic_region,maxmind_continent
0,Afghanistan,AF,Global South,Asia
1,Åland Islands,AX,Global North,Europe
2,Albania,AL,Global North,Europe
3,Algeria,DZ,Global South,Africa
4,American Samoa,AS,Global South,Oceania
...,...,...,...,...
246,Western Sahara,EH,Global South,Africa
247,Yemen,YE,Global South,Asia
248,Zambia,ZM,Global South,Africa
249,Zimbabwe,ZW,Global South,Africa


`wmf.hive.load_csv` allows you to load data in CSV (or TSV) format into the Data Lake.

In [15]:
wmf.hive.load_csv?

Signature:
wmf.hive.load_csv(
    path,
    field_spec,
    db_name,
    table_name,
    create_db=False,
    sep=',',
    headers=True,
)
Docstring:
Upload a CSV (or other delimiter-separated value file) to Data Lake's HDFS,
for use with Hive and other utilities.

`field_spec` specifies the field names and their formats, for the
`CREATE TABLE` statement; for example, `name string, age int, graduated
bool`.

To prevent errors caused by typos, the function will not try to create the
database first unless `create_db=True` is passed.

`headers` gives whether the file has a header row; if it does, the
function strips it before uploading, because Hive treats all rows as
data rows.
File:      ~/wmfdata-python/wmfdata/hive.py
Type:      function


For example, we can use it to load the `users` dataframe we extracted from MariaDB into the Data Lake.

In [43]:
users

,wiki,user_name,user_registration
user_id,,,
13076609,enwiki,Neil Shah-Quinn,2010-09-15 02:10:55
24753008,enwiki,Neil Shah-Quinn (WMF),2015-04-13 15:45:38
1726054,eswiki,Neil Shah-Quinn,2011-02-04 23:06:08
3643219,eswiki,Neil Shah-Quinn (WMF),2015-04-16 17:21:43
1533207,frwiki,Neil Shah-Quinn,2013-04-07 19:35:02
2194938,frwiki,Neil Shah-Quinn (WMF),2015-04-20 17:43:18
311761,arwiki,Neil Shah-Quinn,2010-09-20 16:00:11
981709,arwiki,Neil Shah-Quinn (WMF),2015-04-17 22:09:46
701248,jawiki,Neil Shah-Quinn,2013-01-24 02:55:15


First, we save the dataframe to a CSV file.

In [42]:
users.to_csv("./users.csv")

Then we use `wmf.hive.load_csv`.

In [45]:
wmf.hive.load_csv(
    "./users.csv",
    "user_id INT, wiki STRING, user_name STRING, user_registration TIMESTAMP",
    "default",
    "wmfdata_docs_user"
)

And now the data is available in the Data Lake, so we can access it using Hive, Presto, or Spark.

In [46]:
wmf.presto.run("""
SELECT *
FROM default.wmfdata_docs_user
""")

,user_id,wiki,user_name,user_registration
0,13076609,enwiki,Neil Shah-Quinn,2010-09-15 02:10:55
1,24753008,enwiki,Neil Shah-Quinn (WMF),2015-04-13 15:45:38
2,1726054,eswiki,Neil Shah-Quinn,2011-02-04 23:06:08
3,3643219,eswiki,Neil Shah-Quinn (WMF),2015-04-16 17:21:43
4,1533207,frwiki,Neil Shah-Quinn,2013-04-07 19:35:02
5,2194938,frwiki,Neil Shah-Quinn (WMF),2015-04-20 17:43:18
6,311761,arwiki,Neil Shah-Quinn,2010-09-20 16:00:11
7,981709,arwiki,Neil Shah-Quinn (WMF),2015-04-17 22:09:46
8,701248,jawiki,Neil Shah-Quinn,2013-01-24 02:55:15
9,983069,jawiki,Neil Shah-Quinn (WMF),2015-06-03 15:06:39


Now let's just clean up a little.

In [47]:
import os

os.remove("./users.csv")

wmf.hive.run("""
DROP TABLE default.wmfdata_docs_user
""")

## Utils

Finally, `wmf.utils` has a miscellaneous collection of useful functions.

### `pd_display_all`

`wmf.utils.pd_display_all` disables the limits on how much of a Pandas data frame is displayed in the notebook. This is useful when you have a wide or long data frame that you want to manually inspect. Obviously, be careful; this won't work well if you have thousands of rows or columns.

Normally, Pandas does not show more than 50 characters from a single column.

In [25]:
hive_plan = wmf.hive.run("""
EXPLAIN
SELECT * 
FROM canonical_data.countries
""")

hive_plan

,Explain
0,STAGE DEPENDENCIES:
1,Stage-0 is a root stage
2,
3,STAGE PLANS:
4,Stage: Stage-0
5,Fetch Operator
6,limit: -1
7,Processor Tree:
8,TableScan
9,alias: countries


After using `wmf.utils.pd_display_all`, the full width of the column is displayed (using horizontal scrolling if necessary). 

In [18]:
hive_plan.pipe(wmf.utils.pd_display_all)

,Explain
0,STAGE DEPENDENCIES:
1,Stage-0 is a root stage
2,
3,STAGE PLANS:
4,Stage: Stage-0
5,Fetch Operator
6,limit: -1
7,Processor Tree:
8,TableScan
9,alias: countries


### `insert_code_toggle`

`wmf.utils.insert_code_toggle` inserts some code into the notebook that, in an exported HTML version, will hide the code but insert a button that turn it back on. This is useful for sharing a research notebook with people who mostly just want to see the results rather than what code was used to get them.

In [22]:
wmf.utils.insert_code_toggle?

Signature: wmf.utils.insert_code_toggle()
Docstring:
Outputs a button that will show or hide the code cells in exported HTML
versions of the notebook.
File:      ~/wmfdata-python/wmfdata/utils.py
Type:      function


For an example, see https://analytics.wikimedia.org/published/notebooks/WMF-Language/key-metrics.html By default, you only see a selection of graphs. If you click the "show code" button, the code that generates them appears.

### `df_to_remarkup`

`wmf.utils.df_to_remarkup` converts a Pandas dataframe to a string of [Remarkup](https://secure.phabricator.com/book/phabricator/article/remarkup/), Phabricator's markup language. 

(If you want to create a wikitext or MediaWiki table, use the [Tabulate package](https://github.com/astanin/python-tabulate).)

In [23]:
wmf.utils.df_to_remarkup?

Signature: wmf.utils.df_to_remarkup(df)
Docstring:
Prints a Pandas dataframe as a Remarkup table suitable for pasting into
Phabricator.

Best used via the `pipe`, as in `my_dataframe.pipe(df_to_remarkup)`.
File:      ~/wmfdata-python/wmfdata/utils.py
Type:      function


In [37]:
users

['Neil Shah-Quinn',
 'Neil Shah-Quinn (WMF)',
 'Jimbo Wales',
 'Katherine (WMF)',
 'MIskander-WMF']

In [28]:
users.reset_index().pipe(wmf.utils.df_to_remarkup)

| user_id | wiki | user_name | user_registration | is_staff_account
| ----- | ----- | ----- | ----- | ----- 
| 13076609 | enwiki | Neil Shah-Quinn | 2010-09-15 02:10:55 | 0
| 24753008 | enwiki | Neil Shah-Quinn (WMF) | 2015-04-13 15:45:38 | 1
| 1726054 | eswiki | Neil Shah-Quinn | 2011-02-04 23:06:08 | 0
| 3643219 | eswiki | Neil Shah-Quinn (WMF) | 2015-04-16 17:21:43 | 1
| 1533207 | frwiki | Neil Shah-Quinn | 2013-04-07 19:35:02 | 0
| 2194938 | frwiki | Neil Shah-Quinn (WMF) | 2015-04-20 17:43:18 | 1
| 311761 | arwiki | Neil Shah-Quinn | 2010-09-20 16:00:11 | 0
| 981709 | arwiki | Neil Shah-Quinn (WMF) | 2015-04-17 22:09:46 | 1
| 701248 | jawiki | Neil Shah-Quinn | 2013-01-24 02:55:15 | 0
| 983069 | jawiki | Neil Shah-Quinn (WMF) | 2015-06-03 15:06:39 | 1



### `get_dblist`

`wmf.utils.get_dblist` fetches the contents of a Wikimedia database list (e.g. `wikipedia`, the list of all Wikipedia databases on the cluster).

In [25]:
wmf.utils.get_dblist?

Signature:
wmf.utils.get_dblist(
    dblist_name,
    dblist_path='/srv/mediawiki-config/dblists',
)
Docstring:
Given the name of a dblist (e.g. "wikipedia", "closed", "group0"), return the wiki database names in that list.

To see all the dblists, visit:
https://github.com/wikimedia/operations-mediawiki-config/tree/master/dblists
File:      ~/wmfdata-python/wmfdata/utils.py
Type:      function


This is particularly useful for passing to the `dbs` parameter of `mariadb.run`, so that you can run a query for all those databases.

In [35]:
wikinews = wmf.utils.get_dblist("wikinews")

wmf.mariadb.run(
    """
    SELECT
        DATABASE() AS wiki,
        COUNT(*) AS pages
    FROM page
    """,
    dbs=wikinews
)

,wiki,pages
0,arwikinews,57366
1,bgwikinews,4048
2,bswikinews,4972
3,cawikinews,14437
4,cswikinews,16124
5,dewikinews,61061
6,elwikinews,12726
7,enwikinews,2867153
8,eowikinews,7953
9,eswikinews,47533


### `sql_tuple`

Finally, `wmf.utils.sql_tuple` can turn a list of items into a "tuple" that you can use in an SQL IN clause.

In [27]:
wmf.utils.sql_tuple?

Signature: wmf.utils.sql_tuple(i)
Docstring:
Given a Python iterable, returns a string representation that can be used in an SQL IN
clause.

For example:
> sql_tuple(["a", "b", "c"])
"('a', 'b', 'c')"

single quotes or backslashes. If you encounter this situation, consult the code comments or ask
the maintainers for help.
File:      ~/wmfdata-python/wmfdata/utils.py
Type:      function


For example, this is useful when you have a long list of users and you want to retrieve information about them from MariaDB.

In [48]:
user_names = [
    "Neil Shah-Quinn",
    "Neil Shah-Quinn (WMF)",
    "Jimbo Wales",
    "Katherine (WMF)",
    "MIskander-WMF"
]

user_names = wmf.utils.sql_tuple(user_names)

wmf.mariadb.run(
    f"""
    SELECT
        user_name,
        user_id,
        user_registration
    FROM user
    WHERE user_name IN {user_names}
    """,
    "metawiki",
    date_col="user_registration"
)

,user_name,user_id,user_registration
0,Jimbo Wales,22,NaT
1,Katherine (WMF),5108188,2014-04-14 18:30:18
2,MIskander-WMF,33117416,2021-09-14 15:18:22
3,Neil Shah-Quinn,596810,2010-12-20 02:45:28
4,Neil Shah-Quinn (WMF),7870607,2015-04-15 20:14:12


## Last update

In [29]:
from datetime import datetime

now = datetime.now().isoformat(sep=' ', timespec='minutes')

version = wmf.metadata.version

print(f"Notebook last run at {now} using Wmfdata v{version}.")

Notebook last run at 2022-11-22 02:26 using Wmfdata v1.4.0.
